created on: Tue Jan 14 09:44:36 2020
<br>
Group 7
<br>
@author: V.B., E.G.

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
import h5py
import warnings
warnings.filterwarnings('ignore')
import datetime

# deep learning
import keras
from keras.models import load_model
from keras.applications.imagenet_utils import decode_predictions

In [22]:
time_s = datetime.datetime.now()

In [23]:
project_path = 'C:/Users/emma.grandgirard/Documents/B - Projet Interpromo/'
models_path = project_path + 'Models/'
data_path = 'Data/data_Interpromo2020/Interpromo2020/All Data/ANALYSE IMAGE/'
path_out = project_path + 'pred'

# Choose social network: SEATGURU, SEATMAESTRO, INSTAGRAM
social_net = 'INSTAGRAM'
insta_hashtag = 'airbus'  # if social_net == 'INSTAGRAM'

# Choose images parameters
size = (224, 224)
greys = False

# Read images to predict

In [24]:
def read_img(img_path: str, size: tuple, greys: bool=False) -> np.array:
    
    img_img = Image.open(img_path)
    img_arr = np.array(img_img.resize(size))
    img_arr = img_arr.reshape(1, size[0], size[1], 1 if greys else 3)
    
    return img_arr

In [25]:
# Path
if social_net == 'SEATGURU' or social_net == 'SEATMAESTRO':
    folder = 'IMG ' + social_net + '/'

elif social_net == 'INSTAGRAM':
    folder = social_net + '/' + insta_hashtag + '/'
    
path_pred = project_path + data_path + folder
imgs_names = os.listdir(path_pred)[:100]  # here, only 100 to test

# Init results DataFrame
df = pd.DataFrame(columns=['img', 'view', 'view_proba', 'manufacturer', 'manufacturer_proba', 'type', 'type_proba'])
df['img'] = imgs_names

In [26]:
# Read all images, convert to array
all_imgs_arr = np.array([read_img(path_pred + imgs_names[k], size=(size[0], size[1]), greys=False) 
                         for k in range(len(imgs_names))])  # len(imgs_names)

# Reshape for prediction
all_imgs_arr = all_imgs_arr.reshape(len(imgs_names), size[0], size[1], 1 if greys else 3)
all_imgs_arr.shape

(100, 224, 224, 3)

In [27]:
%run g7_pickle_save_load.ipynb

# Models pipeline

In [28]:
def pred_from_model(img_arr: np.array, models_path: str, model_name: str) -> str:
    
    # Load model and predict
    model, dic_class = load_files_model(path_mod=models_path, mod_name=model_name)
    preds = model.predict(img_arr)
    labels = [np.argmax(preds[k]) for k in range(len(preds))]
    proba_labels = [np.max(preds[k]) for k in range(len(preds))]
    labels = [list(dic_class.keys())[list(dic_class.values()).index(lab)] for lab in labels]
    
    del model, dic_class  # remove model from environment
    
    return labels, proba_labels

## First step: view

In [29]:
view_labels, view_proba_labels = pred_from_model(img_arr=all_imgs_arr, models_path=models_path, model_name='View')

In [30]:
# Fill DataFrame with View labels
df['view'] = view_labels
df['view_proba'] = view_proba_labels

## Exterior

### Manufacturer and type

In [31]:
# If View == 'Ext', predict aircraft type
ind_ext = df[df['view'] == 'Ext'].index
ext_imgs_arr = all_imgs_arr[ind_ext]
ext_labels, ext_proba_labels = pred_from_model(img_arr=ext_imgs_arr, models_path=models_path, model_name='Ext_typ')

# Fill DataFrame with types and manufacturers
df['type'].loc[ind_ext] = ext_labels
df['type_proba'].loc[ind_ext] = ext_proba_labels
df['manufacturer'].loc[ind_ext] = ['Airbus' if 'A' in typ else 'Boeing' for typ in df['type'].loc[ind_ext]]

## Interior

### Manufacturer

In [32]:
# If View == 'Int', predict manufacturer
ind_int = df[df['view'] == 'Int'].index
int_imgs_arr = all_imgs_arr[ind_int]
int_man_labels, int_man_proba_labels = pred_from_model(img_arr=int_imgs_arr, models_path=models_path, model_name='Int_man')

# Fill DataFrame with manufacturers
df['manufacturer'].loc[ind_int] = int_man_labels
df['manufacturer_proba'].loc[ind_int] = int_man_proba_labels

### Airbus type

In [33]:
# If View == 'Int' and manufacturer == 'Airbus', predict type
ind_int_Airbus = df[(df['view'] == 'Int') & (df['manufacturer'] == 'Airbus')].index
int_Airbus_imgs_arr = all_imgs_arr[ind_int_Airbus]
int_Airbus_labels, int_Airbus_proba_labels = pred_from_model(img_arr=int_Airbus_imgs_arr, models_path=models_path, 
                                                             model_name='Int_Airbus')

# Fill DataFrame with types
df['type'].loc[ind_int_Airbus] = int_Airbus_labels
df['type_proba'].loc[ind_int_Airbus] = int_Airbus_proba_labels

### Boeing type

In [34]:
# If View == 'Int' and manufacturer == 'Boeing', predict type
ind_int_Boeing = df[(df['view'] == 'Int') & (df['manufacturer'] == 'Boeing')].index
int_Boeing_imgs_arr = all_imgs_arr[ind_int_Boeing]
int_Boeing_labels, int_Boeing_proba_labels = pred_from_model(img_arr=int_Boeing_imgs_arr, models_path=models_path, 
                                                             model_name='Int_Boeing')

# Fill DataFrame with types
df['type'].loc[ind_int_Boeing] = int_Boeing_labels
df['type_proba'].loc[ind_int_Boeing] = int_Boeing_proba_labels

# Save results as csv

In [35]:
df

,img,view,view_proba,manufacturer,manufacturer_proba,type,type_proba
0,0.jpg,Ext,0.998276,Airbus,NaN,A330,0.65578
1,1.jpg,Ext,0.999475,Boeing,NaN,777,0.648764
2,10.jpg,Ext,0.999881,Airbus,NaN,A320,0.391037
3,100.jpg,Ext,0.999968,Airbus,NaN,A320,0.999999
4,1000.jpg,Ext,0.890084,Airbus,NaN,A321,0.885305
...,...,...,...,...,...,...,...
95,1088.jpg,Ext,0.999116,Boeing,NaN,747,0.99978
96,1089.jpg,Ext,0.999739,Airbus,NaN,A320,0.783124
97,109.jpg,Ext_Int,0.811883,NaN,NaN,NaN,NaN
98,1090.jpg,Meal,0.458014,NaN,NaN,NaN,NaN


In [36]:
# Path to save
if social_net != 'INSTAGRAM':
    path_save = path_out + '/pred_' + social_net + '.csv'
    
else:
    path_save = path_out + '/pred_' + social_net + '_' + insta_hashtag + '.csv'

# Save
df.to_csv(path_or_buf=path_save, sep=';', encoding='utf-8')

In [37]:
print(datetime.datetime.now() - time_s)

0:07:30.075902
